In [3]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import sys, os
# sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
sys.path.append('C:\\Users\\ryamada\\Desktop\\git\\alife_book_src')
import numpy as np
from alifebook_lib.visualizers import MatrixVisualizer

# visualizerの初期化 (Appendix参照)
visualizer = MatrixVisualizer()

# シミュレーションの各パラメタ
SPACE_GRID_SIZE = 256
dx = 0.01
dt = 1
VISUALIZATION_STEP = 8  # 何ステップごとに画面を更新するか。

# モデルの各パラメタ
Du = 2e-5
Dv = 1e-5
f, k = 0.04, 0.06  # amorphous
# f, k = 0.035, 0.065  # spots
# f, k = 0.012, 0.05  # wandering bubbles
# f, k = 0.025, 0.05  # waves
# f, k = 0.022, 0.051 # stripe

# 初期化
u = np.ones((SPACE_GRID_SIZE, SPACE_GRID_SIZE))
v = np.zeros((SPACE_GRID_SIZE, SPACE_GRID_SIZE))
# 中央にSQUARE_SIZE四方の正方形を置く
SQUARE_SIZE = 20
u[SPACE_GRID_SIZE//2-SQUARE_SIZE//2:SPACE_GRID_SIZE//2+SQUARE_SIZE//2,
  SPACE_GRID_SIZE//2-SQUARE_SIZE//2:SPACE_GRID_SIZE//2+SQUARE_SIZE//2] = 0.5
v[SPACE_GRID_SIZE//2-SQUARE_SIZE//2:SPACE_GRID_SIZE//2+SQUARE_SIZE//2,
  SPACE_GRID_SIZE//2-SQUARE_SIZE//2:SPACE_GRID_SIZE//2+SQUARE_SIZE//2] = 0.25
# 対称性を壊すために、少しノイズを入れる
u += np.random.rand(SPACE_GRID_SIZE, SPACE_GRID_SIZE)*0.1
v += np.random.rand(SPACE_GRID_SIZE, SPACE_GRID_SIZE)*0.1

while visualizer:  # visualizerはウィンドウが閉じられるとFalseを返す
    for i in range(VISUALIZATION_STEP):
        # ラプラシアンの計算
        laplacian_u = (np.roll(u, 1, axis=0) + np.roll(u, -1, axis=0) +
                       np.roll(u, 1, axis=1) + np.roll(u, -1, axis=1) - 4*u) / (dx*dx)
        laplacian_v = (np.roll(v, 1, axis=0) + np.roll(v, -1, axis=0) +
                       np.roll(v, 1, axis=1) + np.roll(v, -1, axis=1) - 4*v) / (dx*dx)
        # Gray-Scottモデル方程式
        dudt = Du*laplacian_u - u*v*v + f*(1.0-u)
        dvdt = Dv*laplacian_v + u*v*v - (f+k)*v
        u += dt * dudt
        v += dt * dvdt
    # 表示をアップデート
    visualizer.update(u)



KeyboardInterrupt: 

In [5]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import sys, os
# sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
sys.path.append('C:\\Users\\ryamada\\Desktop\\git\\alife_book_src')
import numpy as np
from alifebook_lib.visualizers import ArrayVisualizer

# visualizerの初期化 (Appendix参照)
visualizer = ArrayVisualizer()

SPACE_SIZE = 600

# CAのバイナリコーディングされたルール (Wolfram code)
RULE = 26

# CAの状態空間
state = np.zeros(SPACE_SIZE, dtype=np.int8)
next_state = np.empty(SPACE_SIZE, dtype=np.int8)

# 最初の状態を初期化
### ランダム ###
state[:] = np.random.randint(2, size=len(state))
### 中央の１ピクセルのみ１、後は０ ###
state[len(state)//2] = 1

while visualizer:  # visualizerはウィンドウが閉じられるとFalseを返す
    # stateから計算した次の結果をnext_stateに保存
    for i in range(SPACE_SIZE):
        # left, center, right cellの状態を取得
        l = state[i-1]
        c = state[i]
        r = state[(i+1)%SPACE_SIZE]
        # neighbor_cell_codeは現在の状態のバイナリコーディング
        # ex) 現在が[1 1 0]の場合
        #     neighbor_cell_codeは 1*2^2 + 1*2^1 + 0*2^0 = 6となるので、
        #     RULEの６番目のビットが１ならば、次の状態は１となるので、
        #     RULEをneighbor_cell_code分だけビットシフトして１と論理積をとる。
        neighbor_cell_code = 2**2 * l + 2**1 * c + 2**0 * r
        if (RULE >> neighbor_cell_code) & 1:
            next_state[i] = 1
        else:
            next_state[i] = 0
    # 最後に入れ替え
    state, next_state = next_state, state
    # 表示をアップデート
    visualizer.update(1-state)


KeyboardInterrupt: 

In [6]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import sys, os
# sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
sys.path.append('C:\\Users\\ryamada\\Desktop\\git\\alife_book_src')
import numpy as np
from alifebook_lib.visualizers import MatrixVisualizer
# import game_of_life_patterns
import chap02.game_of_life_patterns
# visualizerの初期化 (Appendix参照)
visualizer = MatrixVisualizer()

WIDTH = 50
HEIGHT = 50

state = np.zeros((HEIGHT,WIDTH), dtype=np.int8)
next_state = np.empty((HEIGHT,WIDTH), dtype=np.int8)

# 初期化
### ランダム ###
state = np.random.randint(2, size=(HEIGHT,WIDTH), dtype=np.int8)
### game_of_life_patterns.pyの中の各パターンを利用. 左上(2,2)の位置にセットする. ###
pattern = chap02.game_of_life_patterns.OSCILLATOR
# state[2:2+pattern.shape[0], 2:2+pattern.shape[1]] = pattern

while visualizer:  # visualizerはウィンドウが閉じられるとFalseを返す
    for i in range(HEIGHT):
        for j in range(WIDTH):
            # 自分と近傍のセルの状態を取得
            # c: center (自分自身)
            # nw: north west, ne: north east, c: center ...
            nw = state[i-1,j-1]
            n  = state[i-1,j]
            ne = state[i-1,(j+1)%WIDTH]
            w  = state[i,j-1]
            c  = state[i,j]
            e  = state[i,(j+1)%WIDTH]
            sw = state[(i+1)%HEIGHT,j-1]
            s  = state[(i+1)%HEIGHT,j]
            se = state[(i+1)%HEIGHT,(j+1)%WIDTH]
            neighbor_cell_sum = nw + n + ne + w + e + sw + s + se
            if c == 0 and neighbor_cell_sum == 3:
                next_state[i,j] = 1
            elif c == 1 and neighbor_cell_sum in (2,3):
                next_state[i,j] = 1
            else:
                next_state[i,j] = 0
    state, next_state = next_state, state
    # 表示をアップデート
    visualizer.update(1-state) # 1を黒, 0を白で表示する


KeyboardInterrupt: 